In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!unzip '/content/drive/MyDrive/finbox_nlp_test_100k.csv.zip' -d '/content/finbox/'

Archive:  /content/drive/MyDrive/finbox_nlp_test_100k.csv.zip
  inflating: /content/finbox/finbox_nlp_test_100k.csv  
  inflating: /content/finbox/__MACOSX/._finbox_nlp_test_100k.csv  


In [2]:
!pip install --upgrade plotly

     |████████████████████████████████| 23.9 MB 13 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
import pandas as pd
import numpy as np
import csv
import os
import random
import torch

import nltk
from nltk.corpus import stopwords
import csv
import re

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
train_df = pd.read_csv('/content/finbox/finbox_nlp_test_100k.csv')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102689 entries, 0 to 102688
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   created_at  102689 non-null  object
 1   text        102689 non-null  object
 2   is_retweet  102689 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 1.7+ MB


In [6]:
train_df.describe()

,created_at,text,is_retweet
count,102689,102689,102689
unique,97571,102068,1
top,2019-04-29 02:32:36,watching Game of Thrones,False
freq,5,50,102689


In [7]:
train_df.head(10)

,created_at,text,is_retweet
0,2019-04-22 14:53:19,"If my heart can barely handle a break up, I do...",False
1,2019-05-06 15:23:09,10 Interesting Facts About the 'Game of Throne...,False
2,2019-04-16 15:15:18,The 10 Grateful Dead References in Game of Thr...,False
3,2019-04-15 20:28:42,It’s just really interesting that Game of Thro...,False
4,2019-04-29 02:09:12,I AM STRESSED AF WATCHING GAME OF THRONES YAAA...,False
5,2019-04-29 15:32:53,Who loves game of thrones? We here at UDCC can...,False
6,2019-05-20 10:32:30,OST ~ Game of Thrones Season 4 LIMITED Tour Ed...,False
7,2019-05-04 01:59:37,The next time everyone<U+2019>s getting into a...,False
8,2019-05-24 13:49:06,@ComplexSports @Complex @PUSHA_T Aaron Rodgers...,False
9,2019-04-29 06:19:24,game of thrones was LIT https://t.co/hn8P0SlF43,False


In [8]:
train_df['is_retweet'].unique()

array([False])

In [9]:
train_df.head(10)

,created_at,text,is_retweet
0,2019-04-22 14:53:19,"If my heart can barely handle a break up, I do...",False
1,2019-05-06 15:23:09,10 Interesting Facts About the 'Game of Throne...,False
2,2019-04-16 15:15:18,The 10 Grateful Dead References in Game of Thr...,False
3,2019-04-15 20:28:42,It’s just really interesting that Game of Thro...,False
4,2019-04-29 02:09:12,I AM STRESSED AF WATCHING GAME OF THRONES YAAA...,False
5,2019-04-29 15:32:53,Who loves game of thrones? We here at UDCC can...,False
6,2019-05-20 10:32:30,OST ~ Game of Thrones Season 4 LIMITED Tour Ed...,False
7,2019-05-04 01:59:37,The next time everyone<U+2019>s getting into a...,False
8,2019-05-24 13:49:06,@ComplexSports @Complex @PUSHA_T Aaron Rodgers...,False
9,2019-04-29 06:19:24,game of thrones was LIT https://t.co/hn8P0SlF43,False


In [10]:
train_df['text'][0]

'If my heart can barely handle a break up, I don<U+2019>t even want to know what<U+2019>s going to happen to it on the next episode of Game of Thrones'

In [11]:
train_df['created_at'][0]

'2019-04-22 14:53:19'

### **Get Most Common Words in Tweet's Corpus**

Some Insights from Data is:

1. **Total Data is** : 10,2689

2. **Is Retweet column has only**: False Label

 

In [12]:
##let's get the most common_words this will give me a insights off now my dataset spread looks like,
#temp_list is a list containing all the tokens[sperated on the condition based on space]

train_df['temp_list'] = train_df['text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','Count']
temp.style.background_gradient(cmap='Blues')

##What insights I get?

#This data majorly involves game of thrones content, as these words are major use in the tweet's

,Common_words,Count
0,of,120027
1,Game,60771
2,the,57584
3,Thrones,42139
4,to,34354
5,I,34242
6,and,28166
7,a,25491
8,game,23389
9,thrones,22433


In [13]:
train_df.head(10)

,created_at,text,is_retweet,temp_list
0,2019-04-22 14:53:19,"If my heart can barely handle a break up, I do...",False,"[If, my, heart, can, barely, handle, a, break,..."
1,2019-05-06 15:23:09,10 Interesting Facts About the 'Game of Throne...,False,"[10, Interesting, Facts, About, the, 'Game, of..."
2,2019-04-16 15:15:18,The 10 Grateful Dead References in Game of Thr...,False,"[The, 10, Grateful, Dead, References, in, Game..."
3,2019-04-15 20:28:42,It’s just really interesting that Game of Thro...,False,"[It’s, just, really, interesting, that, Game, ..."
4,2019-04-29 02:09:12,I AM STRESSED AF WATCHING GAME OF THRONES YAAA...,False,"[I, AM, STRESSED, AF, WATCHING, GAME, OF, THRO..."
5,2019-04-29 15:32:53,Who loves game of thrones? We here at UDCC can...,False,"[Who, loves, game, of, thrones?, We, here, at,..."
6,2019-05-20 10:32:30,OST ~ Game of Thrones Season 4 LIMITED Tour Ed...,False,"[OST, ~, Game, of, Thrones, Season, 4, LIMITED..."
7,2019-05-04 01:59:37,The next time everyone<U+2019>s getting into a...,False,"[The, next, time, everyone<U+2019>s, getting, ..."
8,2019-05-24 13:49:06,@ComplexSports @Complex @PUSHA_T Aaron Rodgers...,False,"[@ComplexSports, @Complex, @PUSHA_T, Aaron, Ro..."
9,2019-04-29 06:19:24,game of thrones was LIT https://t.co/hn8P0SlF43,False,"[game, of, thrones, was, LIT, https://t.co/hn8..."


In [14]:
fig = px.bar(temp, x="Count", 
             y="Common_words", 
             title='Commmon Words in Tweet', 
             orientation='h', 
             width=700, height=700,
             color='Common_words')
fig.show()

In [15]:
import plotly.express as px
fig = px.treemap(temp, path=['Common_words'], values='Count',title='Tree Of Most Common Words in Tweets')
fig.show()

In [16]:
train_df.head(20)

,created_at,text,is_retweet,temp_list
0,2019-04-22 14:53:19,"If my heart can barely handle a break up, I do...",False,"[If, my, heart, can, barely, handle, a, break,..."
1,2019-05-06 15:23:09,10 Interesting Facts About the 'Game of Throne...,False,"[10, Interesting, Facts, About, the, 'Game, of..."
2,2019-04-16 15:15:18,The 10 Grateful Dead References in Game of Thr...,False,"[The, 10, Grateful, Dead, References, in, Game..."
3,2019-04-15 20:28:42,It’s just really interesting that Game of Thro...,False,"[It’s, just, really, interesting, that, Game, ..."
4,2019-04-29 02:09:12,I AM STRESSED AF WATCHING GAME OF THRONES YAAA...,False,"[I, AM, STRESSED, AF, WATCHING, GAME, OF, THRO..."
5,2019-04-29 15:32:53,Who loves game of thrones? We here at UDCC can...,False,"[Who, loves, game, of, thrones?, We, here, at,..."
6,2019-05-20 10:32:30,OST ~ Game of Thrones Season 4 LIMITED Tour Ed...,False,"[OST, ~, Game, of, Thrones, Season, 4, LIMITED..."
7,2019-05-04 01:59:37,The next time everyone<U+2019>s getting into a...,False,"[The, next, time, everyone<U+2019>s, getting, ..."
8,2019-05-24 13:49:06,@ComplexSports @Complex @PUSHA_T Aaron Rodgers...,False,"[@ComplexSports, @Complex, @PUSHA_T, Aaron, Ro..."
9,2019-04-29 06:19:24,game of thrones was LIT https://t.co/hn8P0SlF43,False,"[game, of, thrones, was, LIT, https://t.co/hn8..."


### **Cleaning Text as it has a lot of links,emoticons I am generally using regex to solve this!**

In [17]:
## A cleaning funtion as in some features we are having special cheracters + emojis, 
## and some part of funtion 

Combined = re.compile('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)') #This will remove every link,symbols,bad_symbols form text

unicodes = re.compile('u\d+') # for unicodes
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    
    if(text!=text):
        return ""
    text = text.lower() # lowercase text

    text = Combined.sub('',text)
    text = unicodes.sub('',text)

    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

clean_train_data = dict()
clean_train_data['text'] = train_df['text'].apply(clean_text)
clean_train_data['is_retweet'] = train_df['is_retweet']

In [18]:
clean_train_data['text'][15]

'wine cheese place game thrones contest bourbon drinks spirits booze imbibe'

In [19]:
clean_train_data_df = pd.DataFrame(clean_train_data)
clean_train_data_df.to_csv("/content/clean_train.csv")

In [25]:
clean_train_data = pd.read_csv('/content/clean_train.csv')

In [26]:
clean_train_data.head(50)

,Unnamed: 0,text,is_retweet
0,0,heart barely handle break dont even want know ...,False
1,1,10 interesting facts game thrones battle winte...,False
2,2,10 grateful dead references game thrones need ...,False
3,3,really interesting game thrones building big r...,False
4,4,stressed af watching game thrones yaaaaallll,False
5,5,loves game thrones udcc cant get enough battle...,False
6,6,ost game thrones season 4 limited tour edition...,False
7,7,next time everyones getting new avengersgame t...,False
8,8,aaron rodgers game thrones extra f602f602,False
9,9,game thrones lit,False


In [27]:
clean_train_data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [28]:
clean_train_data.head(20)

,text,is_retweet
0,heart barely handle break dont even want know ...,False
1,10 interesting facts game thrones battle winte...,False
2,10 grateful dead references game thrones need ...,False
3,really interesting game thrones building big r...,False
4,stressed af watching game thrones yaaaaallll,False
5,loves game thrones udcc cant get enough battle...,False
6,ost game thrones season 4 limited tour edition...,False
7,next time everyones getting new avengersgame t...,False
8,aaron rodgers game thrones extra f602f602,False
9,game thrones lit,False


### **Checking the Word Count and Distribution in Clean Corpus**

In [29]:
clean_train_data['temp_list'] = clean_train_data['text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in clean_train_data['temp_list'] for item in sublist])
temp_clean = pd.DataFrame(top.most_common(20))
temp_clean.columns = ['Common_words','Count']
temp_clean.style.background_gradient(cmap='Blues')

,Common_words,Count
0,game,96573
1,thrones,93501
2,season,12239
3,episode,11864
4,im,10256
5,watch,9950
6,got,7455
7,like,7358
8,gameofthrones,6541
9,dont,6313


Now after cleaning the data by removing stopwords,bad symbols, emoticons some insights we are getting are:

1. *Games Throne* are major in use which tells that most of the tweets are based on games of throne.

2. Tweets are based on Games of Throne with keywords as Episode,Seasons and Some Links.

3. Tweets contains a lot of links majorly the telegram based.

In [30]:
fig = px.bar(temp_clean, x="Count", 
             y="Common_words", 
             title='Commmon Words in Clean-Tweets', 
             orientation='h', 
             width=700, height=700,
             color='Common_words')
fig.show()

## Some good insights

In [31]:
!pip freeze > requirements.txt